In [69]:
from IPython.display import display,HTML,Image
from ipywidgets import interact_manual
import folium
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime
from time import sleep


In [50]:
def download_data():
    url="https://services6.arcgis.com/bdPqSfflsdgFRVVM/arcgis/rest/services/Parking_Violations_2007_2022/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json"
    url="https://services6.arcgis.com/bdPqSfflsdgFRVVM/arcgis/rest/services/a1e239/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json"
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    with open("cuse-parking-tickets.json", "w") as f:
        json.dump(data['features'], f)
        
download_data()

In [93]:
cache = {
    '1000 EILEEN COLLINS BLVD SYRACUSE, NY' : (43.113505702811246,-76.11989651004016)
}

1

In [96]:
def geocode_syracuse(location):
    global cache
    location = f"{location} SYRACUSE, NY".upper()
    lat, lon = cache.get(location, (0, 0))
    if lat == 0 and lon == 0:
        sleep(0.5)
        query_string = {'q': location, 'format': 'json'}
        headers = {"user-agent": "ist256 example mafudge"}
        url = 'https://nominatim.openstreetmap.org/search'
        response = requests.get(url, params=query_string, headers=headers)
        response.raise_for_status()
        geodata = response.json()
        if geodata != []:
            lat, lon = geodata[0]['lat'], geodata[0]['lon']
            cache[location] = (lat, lon)
            return (lat, lon)
        else:
            return (0, 0)
    else:
        return (lat, lon)
# Testing with syracuse, NY 
geocode_syracuse('2300 FAYETTE ST E')
#geocode_syracuse('1000 EILEEN COLLINS BLVD')

('43.048168', '-76.111425')

In [66]:
with open("cuse-parking-tickets.json", "r") as f:
    data = json.load(f)
    tickets = pd.json_normalize(data)

tickets['coords'] = tickets.apply(lambda row: geocode_syracuse(row['attributes.location']), axis=1)

In [60]:
tickets.to_csv("cuse-parking-tickets-geocoded.csv")

In [70]:
tickets = pd.read_csv("cuse-parking-tickets-geocoded.csv")

In [76]:
def get_date(timestamp_in_milliseconds):
    timestamp = timestamp_in_milliseconds / 1000  # Convert to seconds
    dt_object = datetime.utcfromtimestamp(timestamp)
    date_string = dt_object.strftime('%Y-%m-%d %H:%M:%S')
    return date_string

def get_day_of_week(timestamp_in_milliseconds):
    timestamp = timestamp_in_milliseconds / 1000  # Convert to seconds
    dt_object = datetime.utcfromtimestamp(timestamp)
    date_string = dt_object.strftime('%A')
    return date_string

def get_hour(timestamp_in_milliseconds):
    timestamp = timestamp_in_milliseconds / 1000  # Convert to seconds
    dt_object = datetime.utcfromtimestamp(timestamp)
    date_string = dt_object.strftime('%H')
    return date_string


get_date(1673346660000), get_hour(1673346660000), get_day_of_week(1673346660000)

('2023-01-10 10:31:00', '10', 'Tuesday')

In [78]:
tickets['date_string'] = tickets.apply( lambda row: get_date(row['attributes.issued_date']), axis=1)
tickets['hour'] = tickets.apply( lambda row: get_hour(row['attributes.issued_date']), axis=1)
tickets['dayofweek'] = tickets.apply( lambda row: get_day_of_week(row['attributes.issued_date']), axis=1)

In [79]:
tickets

,Unnamed: 0,attributes.OBJECTID,attributes.ticket_number,attributes.issued_date,attributes.location,attributes.description,attributes.status,attributes.amount,geometry.x,geometry.y,coords,date_string,hour,dayofweek
0,0,1,371770D,1672535400000,100 WALTON ST,UNREGSTRD/UNAFFIXD/ALL VTL401-1/403-1,Adjudication Pending,25,NaN,NaN,"('43.048044', '-76.153734')",2023-01-01 01:10:00,01,Sunday
1,1,2,392244D,1672534920000,300 GENESEE ST W,NO STPNG HERE TO CNR/ALL CO15-316(A)6,Fourth Notice,25,-76.155160,43.052247,"('43.0485203', '-76.2427812')",2023-01-01 01:02:00,01,Sunday
2,2,3,392245D,1672534980000,300 GENESEE ST W,NO STPNG HERE TO CNR/ALL CO15-316(A)6,Adjudication Pending,25,-76.155160,43.052247,"('43.0485203', '-76.2427812')",2023-01-01 01:03:00,01,Sunday
3,3,4,380900D,1672582980000,200 STATE ST N,PRKD BLOCKING CRSSWLK/ALL CO15-316 (A)5,Fourth Notice,60,NaN,NaN,"('43.066684', '-76.0765269')",2023-01-01 14:23:00,14,Sunday
4,4,5,389527D,1672584300000,100 BUCKINGHAM AVE,UNINSPECTED/ ALL TIMES VTL 306-B,Adjudication Approved,0,-76.122297,43.033518,"('43.03349135', '-76.12227665')",2023-01-01 14:45:00,14,Sunday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,1996,382141D,1673264280000,411 MONTGOMERY ST,LIMITED PARKING ONLY 9 AM-6 PM CO15-365,Fourth Notice,25,-76.148667,43.046307,"('43.0461782', '-76.1486104')",2023-01-09 11:38:00,11,Monday
1996,1996,1997,362929D,1673237100000,100 HELEN ST,ODD/EVEN PARKING NOV-MAR CO15-318 (A)2,Adjudication Pending,60,NaN,NaN,"('43.063416', '-76.13486')",2023-01-09 04:05:00,04,Monday
1997,1997,1998,393676D,1673233200000,100 WORDEN AVE,ODD/EVEN PARKING NOV-MAR CO15-318 (A)2,Paid In Full,60,NaN,NaN,"('43.072349', '-76.139985')",2023-01-09 03:00:00,03,Monday
1998,1998,1999,393675D,1673233200000,100 WORDEN AVE,ODD/EVEN PARKING NOV-MAR CO15-318 (A)2,Paid In Full,60,NaN,NaN,"('43.072349', '-76.139985')",2023-01-09 03:00:00,03,Monday


In [89]:
pv = pd.read_csv("Parking_Violations_2023.csv")
pv['dayofweek'] = pv['issued_date'].apply(lambda dt: datetime.strptime(dt, "%Y/%m/%d %H:%M:%S+00").strftime("%A"))
pv['hour'] = pv['issued_date'].apply(lambda dt: datetime.strptime(dt, "%Y/%m/%d %H:%M:%S+00").strftime("%H"))

In [90]:
datetime.strptime("2023/01/01 01:10:00+00", "%Y/%m/%d %H:%M:%S+00").strftime("%A")

'Sunday'

In [97]:
pv['coords'] = pv.apply(lambda row: geocode_syracuse(row['location']), axis=1)

In [99]:
pv.to_csv("cuse_parking_violations.csv")